In [13]:
import pprint
import json
import threading
import time
import tweepy
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sys
sys.path.append("../")
from IoTPy.core.stream import Stream, run
from IoTPy.agent_types.sink import sink_element
from IoTPy.agent_types.op import map_window, map_element
from IoTPy.helper_functions.recent_values import recent_values
from IoTPy.helper_functions.print_stream import print_stream
from IoTPy.concurrency.multicore import get_processes
from IoTPy.concurrency.multicore import get_processes_and_procs
from IoTPy.concurrency.multicore import extend_stream, terminate_stream
import ctypes

consumer_key= "tYoizg5kahFcZKQYqckmwP8gR"
consumer_secret = "zCso1VHAT9JIviauLlRXk785d8q2zT6S9jPAtePyRD1WlgjSYH"
access_token= "1273390906293530630-ljM2LpqQGLSCPQphNwjAwgZLfsd5sy"
access_token_secret = "bn2xeSPV8JEDyNoTiCnjO6oA7HxbLoBSADKcQklIuTZNA"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


[nltk_data] Downloading package punkt to /Users/eveblank/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [36]:
class TwitterTrackwordsToStream(tweepy.streaming.StreamListener):

    def __init__(
            self, consumer_key, consumer_secret,
            access_token, access_token_secret,
            trackwords, stream_name, procs):
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.access_token = access_token
        self.access_token_secret = access_token_secret
        self.trackwords = trackwords
        self.stream_name = stream_name
        self.procs = procs
        self.ready = threading.Event()

    def on_error(self, status):

        print(status)

    def on_data(self, data):

        try:
            if data is None: return True
            if 'extended_tweet' in data:
                extend_stream(self.procs, [json.loads(data)], self.stream_name)
            time.sleep(0.0001)
            return True
        except BaseException as e:
            print (' ')
            if not e or str(e) == '':
                print ('No data from Twitter')
            else:
                print("Error on_data from Twitter: %s" % str(e))
                print ("See TwitterTrackwordsToStream.on_data()")
            print (' ')
            sys.exit()

    def setup(self):

        self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
        self.auth.set_access_token(self.access_token, self.access_token_secret)
        self.twitter_stream = tweepy.Stream(self.auth, self)
        self.ready.set()

    def start(self):

        self.twitter_stream.filter(track=self.trackwords, languages = ["en"])

    def get_thread_object(self):
        self.setup()
        return threading.Thread(target=self.start)
    
def filter_stream(tweet, words):
    text = get_text(tweet)
    broken = nltk.tokenize.casual.casual_tokenize(text)
    for i in range (0, len(words)):
        list = words[i]
        for x in range (0, len(list)):
            count = 0
            marker = words[i][x][0]
            if marker == 'p':
                if words[i][x][1] not in text:
                    count+=1
            else:
                if words[i][x][1] not in broken:
                    count+=1
            if count == len(list):
                tweet = {}
    return tweet
        
def get_text(tweet):
    
    if 'extended_tweet' in tweet:
        text = tweet['extended_tweet']['full_text']
    elif 'text' in tweet:
        text = tweet['text']
    else:
        text = str()
    return text

def print_text(tweet):
    
    if 'extended_tweet' in tweet:
        text = tweet['extended_tweet']['full_text']
    elif 'text' in tweet:
        text = tweet['text']
    else:
        text = str()
    print(text)
        
def twitter_to_stream(
        consumer_key, consumer_secret, access_token, access_token_secret,
        trackwords, stream_name, procs):
  
    obj = TwitterTrackwordsToStream(
        consumer_key, consumer_secret, access_token, access_token_secret,
        trackwords, stream_name, procs)
    return obj.get_thread_object()

def twitter_analysis(
        consumer_key, consumer_secret, access_token, access_token_secret,
        trackwords_primary, trackwords_secondary, tweet_analyzer):

    def f(in_streams, out_streams):
        s = Stream('s')
        map_element(filter_stream, in_streams[0], s, words=trackwords_secondary)
        sink_element(print_text, s)

    multicore_specification = [
        [('x', 'x')],
        [{'name': 'p0', 'agent': f, 'inputs':['x'], 'sources':['x']}]]

    processes, procs = get_processes_and_procs(multicore_specification)
    stream_name = 'x'
    source_thread = twitter_to_stream(
        consumer_key, consumer_secret, access_token, access_token_secret,
        trackwords_primary, stream_name, procs)

    procs['p0'].threads = [source_thread]

    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

if __name__ == '__main__':
    twitter_analysis(consumer_key, consumer_secret, 
                     access_token, access_token_secret,
                     trackwords_primary=['russia'], 
                     trackwords_secondary =[[('w', 'vaccine')]], 
                     tweet_analyzer=print_tweets)


RT @EconomicTimes: #Russia on Tuesday became the first country to approve a #coronavirus vaccine, a move that was met with international sk…
"Putin says Russia has approved 'world first' Covid-19 vaccine. But questions over its safety remain..." Well, folks safety exists in the rest of the world, but there was never such thing as safety in Soviet Russia, so it's fine, and they don't understand your concerns really



RT @maggiekb1: They haven’t disclosed research data on this vaccine to the scientific community (at least not last I knew) and the Phase 3…
RT @SAfmnews: Russian President, Vladimir Putin says Russia has given regulatory approval to a locally made coronavirus vaccine, after less…




RT @mitchellreports: On Russia vaccine, @RichardEngel: "The problem is Russia is not releasing a lot of data. Russia says it has this vacci…






RT @business: BREAKING: Vladimir Putin says Russia has registered the world's first Covid-19 vaccine, and discloses that one of his own dau…







Process Process-26:
Traceback (most recent call last):
  File "/Users/eveblank/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/eveblank/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/eveblank/opt/IoTPy-master/IoTPy/concurrency/multicore.py", line 712, in target
    Stream.scheduler.join()
  File "/Users/eveblank/opt/IoTPy-master/IoTPy/core/compute_engine.py", line 255, in join
    self.compute_thread.join()


KeyboardInterrupt: 

  File "/Users/eveblank/opt/anaconda3/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/Users/eveblank/opt/anaconda3/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
